# Dynamic Query Expansion

This is an implementation of the Dynamic Query Expansion Algorithm for Twitter Data as explained in 
"Unsupervised Spatial Event Detection in Targeted Domains with Applications to Civil Unrest Modeling" by Liang Zhao et al.

The implementation is done in python 2.7.6 and it's dependencies are libraries json,nltk,numpy and scipy.

In [16]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import json
import io
import ast
from collections import defaultdict
from nltk.tokenize import TweetTokenizer
import numpy
from scipy.sparse import lil_matrix

Loading Data from a precollected collection of tweets in a json File which is stored in a unicode format

In [17]:
filename='raw_tweets2.json'
data_json = io.open(filename, mode='r', encoding='utf-8').read() #reads in the JSON file
data_python = json.loads(data_json)

In [18]:
#dictionary to capture the frequency of every tweet feature like token,hashtag,url etc.. 
#will be used to find IDF for said features
features_freq=defaultdict(float)

#NLTK tokenizer for tweets which strips handles from the tweet text and reduces repetition of letters
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

Collecting frequencies of features for IDF identity matrix(Df)

In [19]:
count_tweets=0 #Count for number of tweets in Colombia
for line in data_python:
        d = ast.literal_eval(line)
        if d['location']=='CO': #Since the bounding box contains more countries, apart from Colombia
            count_tweets+=1 
            features_freq[d['name']]+=1.0 #adding screen name of tweet sender
            for r in d['hashtags']:
                 features_freq[r['text']]+=1.0 #adding hashtag's frequencies
            for r1 in d['urls']:
                features_freq[r1['expanded_url']]+=1.0 #Adding URL frequencies
            text_tok=unicode(d['text']).lower()
            list_tok=tknzr.tokenize(text_tok)
            for tok in list_tok:
                if tok.isalpha():
                    features_freq[tok]+=1.0

Making a list of unique features to use for populatin adjacency matrix

In [20]:
features_list=[]
for f in features_freq:
    features_list.append(f)

Adding each tweet to a list of tweets and assigning each tweet with corresponding features in a adjacency matrix.
The adjacency matrix is a sparse matrix from scipy(lil_matrix) to account for large feature list and tweets.

In [21]:
tweet_list=[] #List of Tweet IDs 
tweet_dict={} #Dictionary of tweet ID corrsponding to the text for final result

adj_mat1=lil_matrix((len(features_list),count_tweets),dtype=numpy.float) #Adjaceny matrix for Tweet and FeatureList

count=-1
for line1 in data_python:
    d1 = ast.literal_eval(line1)
    if d1['location']=='CO':
        count+=1 
        tweet_list.append(d1['id'])
        tweet_dict[d1['id']]=d1['text']
        for r in d1['hashtags']:
            adj_mat1[features_list.index(r['text']),count]=1
        for r1 in d1['urls']:
            adj_mat1[features_list.index(r1['expanded_url']),count]=1
        adj_mat1[features_list.index(d1['name']),count]=1
        text_tok1=unicode(d1['text']).lower()
        list_tok1=tknzr.tokenize(text_tok1)
        for tok1 in list_tok1:
            if tok1.isalpha():
                 adj_mat1[features_list.index(tok1),count]=1

Populating a tweet-tweet adjacency matrix for modelling a reply relationship  

In [22]:
adj_mat2=lil_matrix((count_tweets,count_tweets),dtype=numpy.float)
for tw in data_python:
    t1=ast.literal_eval((tw))
    if t1['location']=='CO':
        rep_id=t1['reply']
        if rep_id is not None:
            if unicode(rep_id) in tweet_list:
                ind1=tweet_list.index(t1['id'])
                ind2=tweet_list.index(unicode(rep_id))
                adj_mat2[ind1,ind2]=1
                adj_mat2[ind2,ind1]=1

Converting Feature frequencies to a Diagonal Matrix(Df)

In [23]:
idf_arr=numpy.zeros(len(features_list),dtype=numpy.float)
for i in range(0,len(features_list)):
    idf_arr[i]=1.0/features_freq[features_list[i]]
Df=lil_matrix((len(features_list),len(features_list)),dtype=numpy.float)
Df.setdiag(idf_arr)

In [24]:
wt_tweets=numpy.zeros(count_tweets,dtype=numpy.float) #1D Array for tweet weights 
wt_features=numpy.zeros(len(features_list),dtype=numpy.float) #1D array for feature weights

Populating set of Tweets in seed-query. That is every tweet which contains tweets

In [25]:
seed_query=[u'violación',u'UNICEF',u'abuso',u'mujer'] #Seed Query terms

Tr=set()
for i in range(0,len(tweet_list)):
    for j in range(0,len(features_list)):
        if adj_mat1[j,i]==1:
            for sed in seed_query:
                #print sed
                #print features_list[j]
                if sed==features_list[j]:
                    Tr.add(i)
                    wt_tweets[i]=1.0

Printing Initial Set of Seed Queries

In [26]:
for t in Tr:
    print tweet_dict[tweet_list[t]]

Como es posible que si vas con tu mujer, en un SITP con un solo puesto libre, la hagas sentar en el piso para que tu te sientes en la silla!
Hoy NO es el día de las velitas. Es el día de la Virgen Inmaculada, única mujer que ha concebido sin pasar por el p… https://t.co/aPg3tCeqk7
Usted puede ser la niña mas linda, la mujer mas hermosa, pero si usted es o tiene delirios de ñera, no se me puede hacer atractiva
Ella Mi Reina Mi Gran Mujer La Dueña De Mi Mundo Este Amor Qué No… https://t.co/bFUlAbEald
@CaracolRadio DESPUES DE ESCUCHAR  ESTA ENTREVISTA EN CARACOL RADIO  ,   CORROBORO  QUE @SILVIACORZO  ES UNA MUJER  "EXTRAORDINARIA "
Me gusta la mujer que es frentera y no disimula 🎤
"El reggaeton denigra a la mujer"
- una morronga tercermundista que es moza o le acepta la moza al marido 

Lol la ignorancia😩
En San Cristóbal una mujer entrega en adopción a perros y gatos porque tiene que desalojar su casa. Informes en este teléfono 3102346729.
@AngieMmp buena decisión sabia mujer
Es una mie

Defining Functions to find the max weight and minimum weight from set of tweets

In [27]:
def max_wt(s1,wt1,ind):
    wt=-1.0
    for s in s1:
        if wt1[s]>wt:
            wt=wt1[s]
            ind=s
    return wt,ind

def min_wt(s1,wt1,ind):
    wt=5000
    for s in s1:
        if wt1[s]<wt:
            wt=wt1[s]
            ind=s
    return wt,ind

Actual DQE algorithm to swap out the tweets which are lesser weighted from final set of expanded queries

In [28]:
delta=1.0

while(delta>0.0):
    Tr2=set()
    for ct in range(0,count_tweets):
        if ct not in Tr:
            Tr2.add(ct)
    delt=1.0
    while(delt>0.0):
        ind1=-1
        min_w,ind1=min_wt(Tr,wt_tweets,ind1)
        ind2=-1
        max_w,ind2=max_wt(Tr2,wt_tweets,ind2)
        if(max_w>min_w):
            Tr.remove(ind1)
            Tr2.add(ind1)
            Tr.add(ind2)
            Tr2.remove(ind2)
        else:
            delt=0.0
    
    wt_temp=Df.dot(adj_mat1.tocsr())
    wt_features=wt_temp.dot(wt_tweets.transpose()) #Updating features
    wt_tweets=adj_mat1.tocsr().transpose().dot(wt_features)+(0.5*adj_mat2.dot(wt_tweets))
    wt_tweets = wt_tweets / wt_tweets.max(axis=0) #Updating tweets
    m,w=max_wt(Tr2,wt_tweets,-1)
    m1,w1=min_wt(Tr,wt_tweets,-1)
    delta=m-m1

Printing Final results

In [29]:
for t in Tr:
    print tweet_dict[tweet_list[t]]

Como es posible que si vas con tu mujer, en un SITP con un solo puesto libre, la hagas sentar en el piso para que tu te sientes en la silla!
Hoy NO es el día de las velitas. Es el día de la Virgen Inmaculada, única mujer que ha concebido sin pasar por el p… https://t.co/aPg3tCeqk7
Usted puede ser la niña mas linda, la mujer mas hermosa, pero si usted es o tiene delirios de ñera, no se me puede hacer atractiva
Ella Mi Reina Mi Gran Mujer La Dueña De Mi Mundo Este Amor Qué No… https://t.co/bFUlAbEald
@CaracolRadio DESPUES DE ESCUCHAR  ESTA ENTREVISTA EN CARACOL RADIO  ,   CORROBORO  QUE @SILVIACORZO  ES UNA MUJER  "EXTRAORDINARIA "
Me gusta la mujer que es frentera y no disimula 🎤
"El reggaeton denigra a la mujer"
- una morronga tercermundista que es moza o le acepta la moza al marido 

Lol la ignorancia😩
En San Cristóbal una mujer entrega en adopción a perros y gatos porque tiene que desalojar su casa. Informes en este teléfono 3102346729.
@AngieMmp buena decisión sabia mujer
Es una mie